In [7]:
#!pip install kfp
import sys
sys.path.append('../../../igxartifacts/lib')
sys.path.append('../../../pipelines/sdk/python')

In [4]:
import kfp
from kfp import dsl

In [5]:
import os
import sys

# Convert notebooks to python

In [8]:
from igz import v3io_cred, xcp_op, mount_v3io

In [9]:
NOTEBOOKS_DIR = os.getenv('NOTEBOOKS_DIR','/User/GitHub/igxartifacts/location_based_recommendation')
PREP_NB = os.getenv('PREP_NB','02-preprocessing.ipynb')
TRAIN_NB = os.getenv('TRAIN_NB','03-train_prod2vec_skipgram.ipynb')
LGBM_NB = os.getenv('LGBM_NB','04-lgbm.ipynb')
WORK_DIR = os.getenv('WORK_DIR','/v3io/bigdata/lbr')
LBR_PATH  = os.getenv('LBR_PATH','/v3io/bigdata/lbr')

In [33]:
print(NOTEBOOKS_DIR)

/User/GitHub/igxartifacts/location_based_recommendation


In [10]:
%env NOTEBOOKS_DIR={NOTEBOOKS_DIR}
%env PREP_NB={PREP_NB}
%env TRAIN_NB={TRAIN_NB}
%env LGBM_NB={LGBM_NB}

env: NOTEBOOKS_DIR=/User/GitHub/igxartifacts/location_based_recommendation
env: PREP_NB=02-preprocessing.ipynb
env: TRAIN_NB=03-train_prod2vec_skipgram.ipynb
env: LGBM_NB=04-lgbm.ipynb


In [35]:
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{PREP_NB}
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{TRAIN_NB}
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{LGBM_NB}

#!echo {NOTEBOOKS_DIR}/{PREP_NB}

[NbConvertApp] Converting notebook /User/GitHub/igxartifacts/location_based_recommendation/02-preprocessing.ipynb to python
[NbConvertApp] Writing 7785 bytes to /v3io/bigdata/lbr/02-preprocessing.py
[NbConvertApp] Converting notebook /User/GitHub/igxartifacts/location_based_recommendation/03-train_prod2vec_skipgram.ipynb to python
[NbConvertApp] Writing 9076 bytes to /v3io/bigdata/lbr/03-train_prod2vec_skipgram.py
[NbConvertApp] Converting notebook /User/GitHub/igxartifacts/location_based_recommendation/04-lgbm.ipynb to python
[NbConvertApp] Writing 15283 bytes to /v3io/bigdata/lbr/04-lgbm.py


In [11]:
nuclio_deploy = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/deploy/component.yaml')
nuclio_invoke = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/invoker/component.yaml')

In [45]:
def run_step(NAME,CMD,LBR_PATH):
    """Print env vars."""
    cop = dsl.ContainerOp(
        name=NAME,
        image='python:3.6-jessie',
        command=['ls /lbr',CMD],
    )
    cop.apply(v3io_cred())
    cop.apply(mount_v3io(container='bigdata', sub_path=LBR_PATH, mount_path='/lbr'))
    return cop

In [12]:
@dsl.pipeline(
    name='LBR pipeline',
    description='Runnign LBR in kubeflow'
)
def my_pipeline(
   txt='good morningf',
   val=7,
):
    build = nuclio_deploy(url='./02-nuclio-generator.ipynb', name='generator', project='lbr2', tag='0.1')

    

In [13]:
kfp.compiler.Compiler().compile(my_pipeline, 'scratchpipe.yaml')

In [14]:
EXPERIMENT_NAME = 'Scratch pipeline'

In [15]:
client = kfp.Client(namespace='default-tenant')
arguments = {'txt': 'good evening', 'val': 7}
experiment = client.create_experiment(EXPERIMENT_NAME)
run_name = my_pipeline.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, 'scratchpipe.yaml', arguments)